In [ ]:
!pip install -q transformers datasets librosa evaluate jiwer gradio bitsandbytes accelerate

In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.1 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.37.0
    Uninstalling bitsandbytes-0.37.0:
      Successfully uninstalled bitsandbytes-0.37.0


In [ ]:
!pip install -q peft

In [ ]:
from huggingface_hub import list_datasets
from datasets import(load_dataset, DatasetDict, load_from_disk,
                     Audio, IterableDatasetDict, Features, Value,
                     Sequence, Dataset)
import IPython.display as ipd
from functools import partial

In [ ]:
voice_data = load_dataset("clt013/malay-speech-3k-rows-dataset_v2", token=True)
voice_data

README.md:   0%|          | 0.00/458 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/216M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/59.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2544 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/604 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 2544
    })
    test: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 604
    })
})

In [ ]:
from transformers import WhisperProcessor
model_name = "openai/whisper-small"
language = "malay"
task = "transcribe"
processor = WhisperProcessor.from_pretrained(model_name, language=language, task=task)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
def prepare_dataset(batch):
    # load audio
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    # encode target text to label ids
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch



def is_audio_in_length_range(length):
    return 1.0 < length <= 30.0

def prepare_dataset_done():
  dt = voice_data.map(prepare_dataset,remove_columns=voice_data["train"].column_names).with_format("torch")
  dt["train"] = dt["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
  )
  dt["test"] = dt["test"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
  )
  return dt

In [ ]:
voice_data_new = prepare_dataset_done()
voice_data_new = voice_data_new.shuffle(seed=42)
voice_data_new

DatasetDict({
    train: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 2139
    })
    test: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 513
    })
})

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name,  load_in_8bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [ ]:
model.generation_config.language = "malay"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None
model.generation_config.suppress_tokens = []
model.config.use_cache = False

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
import peft
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [ ]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 245,273,856 || trainable%: 1.4429


In [ ]:
!rm -rf "./whisper-small-ft-malay-peft-v1"

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-ft-malay-peft-v2",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-03,
    warmup_steps=100,
    num_train_epochs=3,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    generation_max_length=128,
    # save_steps=1000,
    # eval_steps=1000,
    logging_steps=100,
    report_to=["tensorboard"],
    remove_unused_columns=False,
    label_names=["labels"]
)

In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
import os

# This callback helps to save only the adapter weights and remove the base model weights.
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=voice_data_new["train"],
    eval_dataset=voice_data_new["test"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cp

Step,Training Loss,Validation Loss
100,2.100100,0.840708
200,0.730500,0.787892
300,0.615000,0.740082
400,0.436400,0.712580
500,0.395100,0.677210
600,0.242800,0.664871
700,0.185000,0.642594
800,0.178100,0.633642


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cp

TrainOutput(global_step=804, training_loss=0.6082433417661867, metrics={'train_runtime': 1710.2405, 'train_samples_per_second': 3.752, 'train_steps_per_second': 0.47, 'total_flos': 1.88455405510656e+18, 'train_loss': 0.6082433417661867, 'epoch': 3.0})

In [ ]:
kwargs = {
    "dataset_tags": "clt013/malay-speech-3k-rows-dataset_v2",
    "dataset": "Malay Speech 3k",  # a 'pretty' name for the training dataset
    "language": "ms",
    "model_name": "Whisper Small FT Malay - CLT013",  # a 'pretty' name for your model
    "finetuned_from": model_name,
    "tasks": "automatic-speech-recognition",
}
trainer.push_to_hub(**kwargs)
repo_id = "clt013/whisper-small-ft-malay-peft-v1"
model.push_to_hub(repo_id)

No files have been modified since last commit. Skipping to prevent empty commit.


README.md:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/clt013/whisper-small-ft-malay-peft-v1/commit/d1899b800d25a51ae966dcb5db455aae17809b25', commit_message='Upload model', commit_description='', oid='d1899b800d25a51ae966dcb5db455aae17809b25', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = "clt013/whisper-small-ft-malay-peft-v1"
peft_config = PeftConfig.from_pretrained(peft_model_id)
model_loaded = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)
model_loaded = PeftModel.from_pretrained(model_loaded, peft_model_id)

adapter_config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


adapter_model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

eval_dataloader = DataLoader(voice_data_new["test"], batch_size=8, collate_fn=data_collator)

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
            metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )
            print(step)
            print(decoded_preds)
            print(decoded_labels)
    del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute()
print(f"{wer=}")


  0%|          | 0/65 [00:00<?, ?it/s]<ipython-input-90-d2fde27b1a1a>:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


0
['ah', 'H P Laita H P M', 'tarik tarik', 'sebab huawai pun centre sini kurang kan kalau macam Samsung', 'badminton so lah cantuk badminton', 'kegemaran aku', 'menyalakkan daripada masalah masalah ni berlaku lah macam ah kesihatan kecemanya semua ni sebab dia sangat bahaya lah sebab', 'kemulian boleh beraku dia dalam family']
['ah', 'H H P laptop H P tu', 'tarik tarik', 'sebab Huawei pun centre sini kurang kan kalau macam Samsung', 'badminton sudah tentu', 'kegemaran aku', 'mengelakkan daripada masalah-masalah ni berlakulah macam ah kesihatan kencing manis semua ni sebab dia sangat berbahaya la sebab', 'kemurungan boleh berlaku di dalam family']


  2%|▏         | 1/65 [00:05<06:19,  5.93s/it]

1
['kalau nak makan tu memilih so aku macam tak tahu sangat pasal makanan ni', 'tusai ni kaya dengan kabur hijat protein', 'oh', 'kena ada banyak macam isis itu kan banyak dunia ni', 'makanan ni', 'hm hm okey nanti kalau', 'yang untuk makan makanan makan besar tu kan', 'ah boleh awak tahu bagitahu lah apa sebab ah adik saya terkena demam tersebut apa bila bermain [UNK] bermain di hujan']
['kalau nak makan tu memilih so aku macam tak tahu sangat pasal makanan ni', 'tosai ni kaya dengan karbohidrat protein', 'oh', 'kan ada banyak macam [UNK] situ kan banyak dia punya ni', 'makanan eh', 'um um okey nanti kalau', 'yang untuk makan makanan makan besar tu kan', 'ah boleh awak tahu bagitahu lah apa sebab ah adik saya terkena demam tersebut apabila bermain bermandi hujan']


  3%|▎         | 2/65 [00:11<06:16,  5.97s/it]

...

63
['ah Ness Ness Tata lepas tu bahagian Jet N', 'sekiranya tak dapat kawal', 'pemakanan juga lah tapi saya tidak tahu ah dari mana pemakanan itu ah boleh berlaku nya Covid nineteen lah sebab', 'sedangkan berenang mengambil', 'ha Kuih Baulu aku suka sangat Kuih Baulu ni', 'tak setuap basis [UNK]', 'berdilakan rumah kau kan nak ah nanti', 'aku pun kan kalau boleh nak cari yang apa ni']
['ha nurse nurse datang lepas tu bagi inject kan', 'kiranya tak dapat kawal', 'pemakanan jugalah tapi saya tidak tahu ah dari mana pemakanan itu boleh berlakunya Covid ninteen la sebab', 'sedangkan berenang mengambil', 'ha Kuih Bahulu aku suka sangat Kuih Bahulu ni', '[UNK] situ berbeza sikit', 'pergilah kat rumah kau kan nak ah nanti', 'aku ingat kalau boleh nak cari yang apa ni']


 98%|█████████▊| 64/65 [05:58<00:04,  4.50s/it]

64
['okey okey okey ah']
['okay okay okay ah']


100%|██████████| 65/65 [06:00<00:00,  5.54s/it]

wer=32.29689067201605
